In [94]:
import numpy as np

print("Loading data ...")

data_root = 'timit_11'
train = np.load(data_root + '/train_11.npy')
train_label = np.load(data_root + '/train_label_11.npy')
test = np.load(data_root + '/test_11.npy')

print("Size of training data: {}".format(train.shape))
print("Size of testing data: {}".format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [96]:
state = np.random.get_state()
np.random.shuffle(train)
np.random.set_state(state)
np.random.shuffle(train_label)

In [97]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y = None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
    
    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [98]:
VAL_RATIO = 0.01

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]

print("Size of training set: {}".format(train_x.shape))
print("Size of validation set: {}".format(val_x.shape))

Size of training set: (1217632, 429)
Size of validation set: (12300, 429)


In [99]:
from torch.utils.data import DataLoader

BATCH_SIZE = 512

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)

train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle = False)

In [100]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

104

In [101]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer0_bn = nn.BatchNorm1d(429)
        
        self.layer1 = nn.Linear(429, 2048)
        self.layer1_bn = nn.BatchNorm1d(2048)
        
        self.layer2 = nn.Linear(2048, 2048)
        self.layer2_bn = nn.BatchNorm1d(2048)
        
        self.layer3 = nn.Linear(2048, 2048)
        self.layer3_bn = nn.BatchNorm1d(2048)
        
        self.layer4 = nn.Linear(2048, 2048)
        self.layer4_bn = nn.BatchNorm1d(2048)
        
        self.layer5 = nn.Linear(2048, 2048)
        self.layer5_bn = nn.BatchNorm1d(2048)
        
        self.out = nn.Linear(2048, 39)
        
        self.dropoutInput = nn.Dropout(p = 0.2)
        self.dropout = nn.Dropout(p = 0.5)
        
        self.act_fn = nn.ReLU()
    
    def forward(self, x):
        x = self.layer0_bn(x)
        x = self.dropoutInput(x)
        
        x = self.layer1(x)
        x = self.layer1_bn(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer2(x)
        x = self.layer2_bn(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer3(x)
        x = self.layer3_bn(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer4(x)
        x = self.layer4_bn(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer5(x)
        x = self.layer5_bn(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.out(x)
        return x

In [102]:
def get_device():
    return 'cuda' if torch.cuda.is_available() == True else 'cpu'

In [103]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [104]:
same_seeds(0)

device = get_device()
print("DEVICE: {}".format(device))

num_epochs = 100
learning_rate = 0.001
early_stop = 20

model_path = './model.ckpt'
model = Classifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

DEVICE: cuda


In [105]:
best_acc = 0.0
early_stop_cnt = 0

for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0
    
    model.train()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
#         reg_loss = 0
#         for param in model.parameters():
#             reg_loss += torch.sum(param ** 2)
#         loss += 0.00075 * reg_loss
        _, train_preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += (train_preds.cpu() == labels.cpu()).sum().item()
    
    if len(val_set) > 0:
        model.eval()
        
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
#                 reg_loss = 0
#                 for param in model.parameters():
#                     reg_loss += torch.sum(param ** 2)
#                 loss += 0.00075 * reg_loss
                _, val_preds = torch.max(outputs, 1)
                
                val_loss += loss.item()
                val_acc += (val_preds.cpu() == labels.cpu()).sum().item()
            
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                 epoch + 1, num_epochs, train_acc / len(train_set), train_loss / len(train_loader), val_acc / len(val_set), val_loss / len(val_loader)
             ))
        
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                early_stop_cnt = 0
                print('saving model with acc {:.3f}'.format(best_acc / len(val_set)))
            else:
                early_stop_cnt += 1
                if(early_stop_cnt >= early_stop):
                    break
                
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epochs, train_acc / len(train_set), train_loss / len(train_loader)
        ))
        
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

[001/100] Train Acc: 0.568132 Loss: 1.392063 | Val Acc: 0.670976 loss: 1.028025
saving model with acc 0.671
[002/100] Train Acc: 0.628680 Loss: 1.167372 | Val Acc: 0.698618 loss: 0.926482
saving model with acc 0.699
[003/100] Train Acc: 0.650225 Loss: 1.091552 | Val Acc: 0.712927 loss: 0.868010
saving model with acc 0.713
[004/100] Train Acc: 0.663834 Loss: 1.042040 | Val Acc: 0.723415 loss: 0.821924
saving model with acc 0.723
[005/100] Train Acc: 0.674101 Loss: 1.005106 | Val Acc: 0.731951 loss: 0.789527
saving model with acc 0.732
[006/100] Train Acc: 0.681501 Loss: 0.977676 | Val Acc: 0.739512 loss: 0.766864
saving model with acc 0.740
[007/100] Train Acc: 0.687464 Loss: 0.955807 | Val Acc: 0.748049 loss: 0.745484
saving model with acc 0.748
[008/100] Train Acc: 0.692833 Loss: 0.937016 | Val Acc: 0.754715 loss: 0.727212
saving model with acc 0.755
[009/100] Train Acc: 0.696895 Loss: 0.922548 | Val Acc: 0.757724 loss: 0.720874
saving model with acc 0.758
[010/100] Train Acc: 0.70078

In [106]:
test_set = TIMITDataset(test)
test_loader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = False)

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [107]:
predicts = []
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_preds = torch.max(outputs, 1)
        
        for y in test_preds.cpu().numpy():
            predicts.append(y)

In [108]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, p in enumerate(predicts):
        f.write('{},{}\n'.format(i, p))

In [109]:
len(predicts)

451552